# ARCH vs GARCH: Parsimony Comparison

Comparison of ARCH(5) vs GARCH(1,1) conditional volatility showing parsimony advantage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_quiz2_arch_vs_garch/TSA_ch5_quiz2_arch_vs_garch.ipynb)

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [ ]:
# Style constants
BLUE = '#1A3A6E'
RED = '#DC3545'
GREEN = '#2E7D32'
ORANGE = '#E67E22'
PURPLE = '#7B2D8E'

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.size': 11,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
})

In [ ]:
np.random.seed(123)
n = 300
z = np.random.standard_normal(n)

# ARCH(5) simulation
omega_a = 0.0001
alphas = [0.15, 0.12, 0.10, 0.08, 0.05]
sigma2_arch = np.full(n, omega_a / (1 - sum(alphas)))
eps_arch = np.zeros(n)
eps_arch[:5] = z[:5] * np.sqrt(sigma2_arch[0])

for t in range(5, n):
    sigma2_arch[t] = omega_a + sum(a * eps_arch[t-i-1]**2 for i, a in enumerate(alphas))
    eps_arch[t] = np.sqrt(sigma2_arch[t]) * z[t]

# GARCH(1,1) simulation (similar persistence with fewer params)
omega_g, alpha_g, beta_g = 0.00002, 0.10, 0.85
sigma2_garch = np.full(n, omega_g / (1 - alpha_g - beta_g))
eps_garch = np.zeros(n)
eps_garch[0] = z[0] * np.sqrt(sigma2_garch[0])

for t in range(1, n):
    sigma2_garch[t] = omega_g + alpha_g * eps_garch[t-1]**2 + beta_g * sigma2_garch[t-1]
    eps_garch[t] = np.sqrt(sigma2_garch[t]) * z[t]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(np.sqrt(sigma2_arch), color=BLUE, linewidth=1)
ax1.set_title('ARCH(5): 5 parameters', fontsize=11, fontweight='bold', color=BLUE)
ax1.set_ylabel('Conditional Volatility ($\\sigma_t$)', fontsize=10)
ax1.set_xlabel('Time', fontsize=10)
ax1.text(0.05, 0.92, '$\\sigma_t^2 = \\omega + \\sum_{i=1}^{5} \\alpha_i \\varepsilon_{t-i}^2$',
         transform=ax1.transAxes, fontsize=9, color=BLUE,
         bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=BLUE, alpha=0.8))

ax2.plot(np.sqrt(sigma2_garch), color=RED, linewidth=1)
ax2.set_title('GARCH(1,1): 2 parameters', fontsize=11, fontweight='bold', color=RED)
ax2.set_ylabel('Conditional Volatility ($\\sigma_t$)', fontsize=10)
ax2.set_xlabel('Time', fontsize=10)
ax2.text(0.05, 0.92, '$\\sigma_t^2 = \\omega + \\alpha \\varepsilon_{t-1}^2 + \\beta \\sigma_{t-1}^2$',
         transform=ax2.transAxes, fontsize=9, color=RED,
         bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=RED, alpha=0.8))

fig.suptitle('ARCH vs GARCH: Parsimony', fontsize=12, fontweight='bold', color=BLUE, y=1.02)

fig.legend(['ARCH(5) — needs many lags', 'GARCH(1,1) — captures persistence with 2 params'],
           loc='lower center', ncol=2, frameon=False, fontsize=9,
           bbox_to_anchor=(0.5, -0.05))

fig.tight_layout()
fig.subplots_adjust(bottom=0.12)
plt.show()